# Optimize custom model metrics with hyperparameter tuning

Author: Zeeshan Arif and Oleksandr Ruppelt

Version date: December 2025

This notebook demonstrates how to improve DataRobot models using custom loss functions and advanced hyperparameter tuning.

In many real-world business problems, standard metrics like RMSE or Accuracy do not fully represent the true business cost. For example, in CLV prediction, overpredicting loss-making customers or underpredicting high-value customers can directly impact revenue and retention strategy. Similarly, classification models often need custom objectives such as maximizing recall at specific thresholds or minimizing false negatives.

By creating a custom metric and tuning models using that metric, you ensure the model is optimized for business value, not just statistical performance. This approach is essential when:

- Business costs are not symmetrical (over-prediction and under-prediction have different impacts)
- False negatives and false positives carry different risk levels 
- Revenue-driven metrics matter more than standard ML scores
- Domain rules must be incorporated into optimization

## What this notebook does

- Connects to your DataRobot project
- Supports both classification (binary & multiclass) and regression projects
- Selects top-performing models from the leaderboard for optimization
- Uses Bayesian optimization (HyperOpt) to find better hyperparameters
- Evaluates models using your custom loss function and tunes the models with hyperparameters
- Provides a comparison of original vs. optimized performance

### Project validation rule

This notebook requires the DataRobot project to be created with Cross-Validation (CV) as the validation method. This is because the tuning logic evaluates models using cross-validation predictions (excluding holdout).

## How HyperOpt works

HyperOpt is a Bayesian optimization framework that efficiently searches the hyperparameter space. It uses Tree-structured Parzen Estimators (TPE) to model the performance landscape. Instead of random or grid search, it learns from past evaluations to suggest better hyperparameter combinations. This leads to faster convergence and better model performance with fewer iterations.

## How to use

1. **Download the notebook**  
   Clone or download the notebook from your repository or workspace.

2. **Create a codespace (optional but recommended)**  
   Create codespace inside DataRobot and upload this notebook.

3. **Install required libraries**  
   All dependencies are listed in the `requirements.txt` file. Run the following command in a terminal or a notebook cell:
   ```bash
   pip install -r requirements.txt
   ```
   Alternatively, a `pip install` cell is already provided in the notebook, simply run that cell.

4. **Update the configuration section**  
   Set your `PROJECT_ID`, `DR_API_TOKEN`, `DR_ENDPOINT`, and `FALLBACK_TRAINING_PATH` as needed.

5. **Run all cells sequentially**  
   Execute each cell from top to bottom to ensure proper initialization and execution.

6. **Review results**  
   Compare original vs optimized model performance in the final output table and visualizations.


# Install & import required libraries

## Install libraries

In [ ]:
!pip install pandas
!pip install numpy
!pip install datetime
!pip install warnings
!pip install time
!pip install tqdm
!pip install itertools
!pip install plotly
!pip install graphviz
!pip install seaborn
!pip install datarobot
!pip install scikit-learn
!pip install hyperopt

## Import libraries

In [51]:
import datetime as dt
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")
from itertools import product
import os
import time

# DataRobot
import datarobot as dr
import graphviz

# Optimization and ML metrics
from hyperopt import fmin, hp, STATUS_OK, tpe, Trials

# Visualization
import plotly.express as px
import seaborn as sns

# Sklearn
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    mean_absolute_error,
    mean_squared_error,
    precision_score,
    r2_score,
    recall_score,
    roc_auc_score,
)
import tqdm
from tqdm import tqdm

# ----- Set display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 50)
sns.set_style("whitegrid")
from IPython.display import display, Markdown

print("***************** All libraries imported successfully!")

***************** All libraries imported successfully!


## Custom metrics library

This cell defines a set of custom evaluation metrics for both classification and regression tasks. These metrics can be used to assess model performance beyond standard metrics like accuracy or RMSE. Define your custom metrics below.

### Binary classification metrics

- `my_cost_score`: Simplified version of `my_cost` returning only the cost value.

### Regression metrics

- `clv_asymmetry_loss`: A custom asymmetric loss function designed for Customer Lifetime Value (CLV) prediction. It penalizes:
  - Overprediction of negative CLV (risk of overspending) with a configurable `over_penalty`
  - Underprediction of positive CLV (missed revenue) with a configurable `under_penalty`
  - This allows for business-aware tuning of regression models.

Custom metrics are especially powerful when standard metrics fail to capture business impact—use them to align model performance with real-world outcomes.

In [52]:
# ===============================================================
# CUSTOM METRICS LIBRARY
# ===============================================================
# You can define your own metrics directly in this cell.


# -------------------- Binary classification metrics --------------------


def my_cost(y_true, y_pred_proba):
    """
    Calculates a custom cost metric based on True Negative Rate (TNR) at the highest threshold
    that achieves 100% recall.

    This function iterates over thresholds from 0.00 to 0.99 to find the one that:
    - Achieves perfect recall (i.e., all positive samples are correctly identified)
    - Maximizes the true negative rate (TNR) among those thresholds

    The cost is defined as: cost = 1 - best TNR"""

    thresholds = np.arange(0.00, 1.0, 0.01)
    best_tnr = -1
    best_thresh = None
    best_recall = None
    best_tn = best_fp = best_fn = best_tp = 0

    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

        if recall >= 1.0:  # Only consider thresholds that give 100% recall
            tnr = tn / (tn + fp) if (tn + fp) > 0 else 0.0
            if tnr > best_tnr:
                best_tnr = tnr
                best_thresh = threshold
                best_recall = recall
                best_tn, best_fp, best_fn, best_tp = tn, fp, fn, tp

    if best_tnr == -1:
        cost = 1.0
        best_thresh = 0.0
        best_tnr = 0.0
        best_recall = 0.0
        best_tn = best_fp = best_fn = best_tp = 0
    else:
        cost = 1 - best_tnr

    return cost, best_thresh, best_recall, best_tnr, best_tn, best_fp, best_fn, best_tp


def my_cost_score(y_true, y_pred_proba):
    cost, *_ = my_cost(y_true, y_pred_proba)
    return cost


# -------------------- Regression metrics --------------------
def clv_asymmetry_loss(y_true, y_pred, over_penalty=2.0, under_penalty=3.0):
    """
    Custom asymmetric loss for CLV:
    - Overpredicting negative CLV is risky (over_penalty)
    - Underpredicting positive CLV misses revenue (under_penalty)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    diff = y_pred - y_true
    penalty = np.where(
        (y_true < 0) & (diff > 0),
        over_penalty * np.abs(diff),  # risky overpredict
        np.where((y_true > 0) & (diff < 0), under_penalty * np.abs(diff), np.abs(diff)),
    )
    return np.mean(penalty)


print(f">>>  Custom Metrics Library loaded successfully")

>>>  Custom Metrics Library loaded successfully


# Helper functions 

## DataRobot helper and universal tuning functions

Below are some of the DataRobot API features used in this notebook.

- Hyperparameter utilities: Extract and format tunable parameters from DataRobot blueprints using `parameters_to_df()` and `get_current_hyperparams()`.
- Project type detection: Automatically identify if a project is binary, multiclass, or regression with `detect_project_type()`.
- Training data and predictions: Fetch training data and predictions with fallback support using `get_training_data()` and `get_or_request_training_predictions()`.
- Search space retrieval: Dynamically assign hyperparameter search spaces based on model names using `get_search_space()`.


In [53]:
# ***************************************************************************************************
#  DataRobot Helper and Universal Tuning Functions  (Supports both Classification and Regression)
# ***************************************************************************************************


def parameters_to_df(params, keep_duplicates=False):
    """
    Return tunable parameters for all blueprint steps from a given modeling blueprint.
    """
    params = params["tuning_parameters"]
    dat_param = pd.DataFrame.from_dict(params)
    dat_param = pd.concat(
        [dat_param, dat_param["constraints"].apply(lambda x: constraints_unfold(x))],
        axis=1,
    )
    dat_param.drop(columns=["constraints"], inplace=True)
    dat_param["parameter_name_type"] = dat_param["parameter_name"] + "_" + dat_param["param_type"]
    res = dat_param[
        [
            "task_name",
            "parameter_name",
            "parameter_name_type",
            "current_value",
            "default_value",
            "param_type",
            "supports_grid_search",
            "min",
            "max",
            "values",
            "parameter_id",
        ]
    ].sort_values("parameter_name_type")
    if keep_duplicates is not None:
        res.drop_duplicates(subset="parameter_name_type", keep=keep_duplicates, inplace=True)
    return res.reset_index(drop=True).sort_values(by="task_name")


# ***************************************************************************************************


def get_or_request_training_predictions(
    model: dr.Model, data_subset: dr.enums.DATA_SUBSET
) -> pd.DataFrame:
    """
    Returns the training predictions for a given model and data subset.
    """
    try:
        training_prediction_job: dr.Job = model.request_training_predictions(
            data_subset=data_subset
        )
        training_prediction_job.wait_for_completion()
        training_predictions = training_prediction_job.get_result_when_complete()
    except dr.errors.ClientError as e:
        if e.status_code == 422:
            training_predictions = [
                tp
                for tp in dr.TrainingPredictions.list(model.project_id)
                if tp.data_subset == data_subset and tp.model_id == model.id
            ][0]
        else:
            raise e
    return training_predictions.get_all_as_dataframe()


# ***************************************************************************************************


def constraints_unfold(c_dict):
    """
    Unfold nested JSON constraints from parameter tuning.
    """
    ser_ = pd.Series(list(c_dict.values())[0])
    ser_["param_type"] = list(c_dict.keys())[0]
    return ser_


# ***************************************************************************************************


def get_current_hyperparams(model, task_filter=None):
    """
    Fetch current hyperparameter details for a given model.
    """
    try:
        tunable_params = model.get_advanced_tuning_parameters()
        params_df = parameters_to_df(tunable_params, keep_duplicates="first")

        params_df.columns = [c.lower().strip() for c in params_df.columns]
        cols_to_keep = [
            "task_name",
            "parameter_name",
            "parameter_name_type",
            "current_value",
            "default_value",
            "param_type",
            "min",
            "max",
        ]
        params_df = params_df[[c for c in cols_to_keep if c in params_df.columns]]
        return params_df

    except Exception as e:
        print(f">>>>>>>>>>>>>> Error fetching params for model {model.id}: {e}")
        return pd.DataFrame()


# ***************************************************************************************************


def get_search_space(model_name):
    """
    Return search space for a given model name.
    (Keep as it is; fallback if not found.)
    """
    for key in SEARCH_SPACES.keys():
        if key.lower() in model_name.lower():
            print(f" Search space found for model: {key}")
            return SEARCH_SPACES[key]

    print(f" Using default search space for model: {model_name}")
    return {
        "n_estimators": hp.quniform("n_estimators", 100, 500, 50),
        "max_depth": hp.quniform("max_depth", 3, 10, 1),
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.3),
    }


# ***************************************************************************************************
# Universal Metric Function (New)
# ***************************************************************************************************

# ***************************************************************************************************
# Detect Project Type
# ***************************************************************************************************


def detect_project_type(proj):
    """Detects if project is binary, multiclass, or regression."""
    if proj.target_type in ["Binary"]:
        return "binary"
    elif proj.target_type in ["Regression"]:
        return "regression"
    else:
        return "multiclass"


# ***************************************************************************************************
# Get Training Data Helper
# ***************************************************************************************************

# Optimization logic

## `tune_and_compare_model()` 

This function performs custom metric-based hyperparameter tuning using HyperOpt for DataRobot models.

### What it does

- Supports only binary classification and regression projects.
- Uses user-defined global variables:
  - `USER_CUSTOM_METRIC`, `METRIC_FUNCTION`, `METRIC_DIRECTION`, `METRIC_INPUT_TYPE`.

### Key steps

- Helper: `detect_prob_col()`  
  Identifies the correct probability column for binary classification predictions.

- Helper: `get_output()`  
  Formats model predictions based on project type and metric input (label or probability).

- Load training data  
  Uses `get_training_data()` and extracts project metadata like target name and type.

- Validate metric configuration  
  Prints the selected custom metric name, function, direction, and input type.

- Evaluate base model  
  Fetches training predictions for the base model and computes the baseline metric score.

- Define `objective()` function  
  - Starts a tuning session on the base model.
  - Applies parameter values from HyperOpt.
  - Trains a new model and evaluates it using the custom metric.
  - Computes performance delta vs. base model.
  - Returns results to HyperOpt.

- Run HyperOpt  
  Executes Bayesian optimization (`tpe.suggest`) over the defined search space for `max_evals` trials.

- Build final comparison table  
  - Aggregates all trials and their metrics.
  - Adds the base model as trial 0.
  - Sorts models by metric (ascending or descending based on `METRIC_DIRECTION`).
  - Returns a final DataFrame comparing all tuned models.

Use this function to automate model tuning with domain-specific metrics and compare improvements over the base model.

In [54]:
## new one :


# **************************************************************************************************
def tune_and_compare_model(proj, base_model_id, search_space, max_evals, fallback_path=None):
    """
    Universal model tuning function (classification + regression) using HyperOpt.

    Performs custom metric-based hyperparameter optimization,
    evaluates the base model and tuned models, and returns a clean comparison table.

    Returns
    -------
    pd.DataFrame
        Columns:
            trial, model_id, model_name, type, USER_CUSTOM_METRIC, delta, status, + tuned parameters
    """

    # -------------------------------------------------------------
    # 🔹 ------------Loading training data and project info
    # -------------------------------------------------------------
    train_df = get_training_data(proj, fallback_path=fallback_path)
    target_name = proj.target
    project_type = detect_project_type(proj)

    if project_type not in ["binary", "regression"]:
        raise RuntimeError(f"XX Unsupported project type: {project_type}")

    print(f"1. Training data shape: {train_df.shape}")
    print(f"2. Target variable: {target_name}")
    print(f"3. Project Type: {project_type}")
    print("=" * 80)

    if "row_id" not in train_df.columns:
        train_df = train_df.reset_index().rename(columns={"index": "row_id"})

    # -------------------------------------------------------------
    # 🔹 Base model & metric setup
    # -------------------------------------------------------------
    base_model = dr.Model.get(proj.id, base_model_id)
    print(f"XX Starting tuning for: {base_model.model_type} | ID: {base_model_id}")
    print(f"XXX Custom Metric: {USER_CUSTOM_METRIC}")
    print(f"XXXX  Metric Function: {METRIC_FUNCTION.__name__}")
    print(f"XXXXX  Direction: {METRIC_DIRECTION}")
    print("=" * 80)

    # -------------------------------------------------------------
    # 🔹 -------------Evaluating base model
    # -------------------------------------------------------------
    preds = get_or_request_training_predictions(base_model, dr.enums.DATA_SUBSET.ALL)
    merged = train_df.merge(preds, on="row_id", how="inner")
    cv = merged[merged["partition_id"] != "Holdout"]

    y_true = cv[target_name].values

    # --- get_output auto-handles regression/classification
    def get_output(df):
        if project_type == "binary":
            prob_col = "class_1" if "class_1" in df.columns else "prediction"
            y_pred = df[prob_col].values
            return (y_pred >= 0.5).astype(int) if METRIC_INPUT_TYPE == "label" else y_pred
        elif project_type == "regression":
            if "prediction" not in df.columns:
                raise KeyError("xxxxxxxxxxx No 'prediction' column found for regression.")
            return df["prediction"].values

    y_pred = get_output(cv)
    base_loss = float(METRIC_FUNCTION(y_true, y_pred))

    print(f"X Base {USER_CUSTOM_METRIC}: {base_loss:.6f}")
    BASE_METRICS = {"metric": base_loss}

    # -------------------------------------------------------------
    # 🔹 -----------Objective Function for HyperOpt
    # -------------------------------------------------------------
    def objective(params):
        print(f"*********** Trying params: {params}")
        try:
            tune_session = base_model.start_advanced_tuning_session()
            task_names = tune_session.get_task_names()

            algo_task = [
                t
                for t in task_names
                if any(k in t.lower() for k in ["classifier", "regressor", "model", "prediction"])
            ]
            algo_task = algo_task[-1] if algo_task else task_names[-1]

            # --- ApplyING parameters safely
            for param, value in params.items():
                value = (
                    int(value) if isinstance(value, float) and value.is_integer() else float(value)
                )
                try:
                    tune_session.set_parameter(
                        task_name=algo_task, parameter_name=param, value=value
                    )
                except Exception as e:
                    print(f"XXXXXXXXXX Skipping param {param}: {e}")

            # --- Run tuning job
            job = tune_session.run()
            tuned_model = job.get_result_when_complete()

            # --- Evaluate tuned model
            preds_tuned = get_or_request_training_predictions(tuned_model, dr.enums.DATA_SUBSET.ALL)
            merged_tuned = train_df.merge(preds_tuned, on="row_id", how="inner")
            cv_tuned = merged_tuned[merged_tuned["partition_id"] != "Holdout"].copy()

            y_true_tuned = cv_tuned[target_name].values
            y_pred_tuned = get_output(cv_tuned)
            tuned_loss = float(METRIC_FUNCTION(y_true_tuned, y_pred_tuned))

            # --- Compute delta
            delta = tuned_loss - base_loss if METRIC_DIRECTION == "max" else base_loss - tuned_loss
            print(f"XXXXXXXXXXXXXX Tuned {USER_CUSTOM_METRIC}: {tuned_loss:.6f} | Δ = {delta:.6f}")

            return {
                "loss": tuned_loss,  # required by HyperOpt
                "status": STATUS_OK,
                "model_id": tuned_model.id,
                "model_name": base_model.model_type,
                "type": "Tuned",
                USER_CUSTOM_METRIC: tuned_loss,
                "delta": delta,
                **{f"param_{k}": v for k, v in params.items()},
            }

        except Exception as e:
            print(f"XXXXXXXXXXXXX Error during tuning trial: {e}")
            return {"loss": 9999.0, "status": "failed", "type": "Tuned"}

    # -------------------------------------------------------------
    # 🔹 --------Running HyperOpt
    # -------------------------------------------------------------
    trials = Trials()
    best = fmin(
        fn=objective, space=search_space, algo=tpe.suggest, max_evals=max_evals, trials=trials
    )
    print("\nxxxxxxxxxxxxxxx Best Params:", best)

    # -------------------------------------------------------------
    # 🔹------------ Collecting all trial results
    # -------------------------------------------------------------
    results = []
    for i, t in enumerate(trials.trials):
        res = t["result"]
        if "loss" not in res or res["loss"] == 9999.0:
            continue
        results.append(
            {
                "trial": i + 1,
                "model_id": res.get("model_id"),
                "model_name": res.get("model_name"),
                "type": res.get("type", "Tuned"),
                USER_CUSTOM_METRIC: res.get(USER_CUSTOM_METRIC, res.get("loss")),
                "delta": res.get("delta", 0.0),
                "status": res.get("status", "ok"),
                **{k: v for k, v in res.items() if k.startswith("param_")},
            }
        )

    df_tuned = pd.DataFrame(results)

    # -------------------------------------------------------------
    # 🔹----------------- Add Base Model Row
    # -------------------------------------------------------------
    base_row = pd.DataFrame(
        [
            {
                "trial": 0,
                "model_id": base_model.id,
                "model_name": base_model.model_type,
                "type": "Original",
                USER_CUSTOM_METRIC: base_loss,
                "delta": 0.0,
                "status": "ok",
            }
        ]
    )

    # -------------------------------------------------------------
    # 🔹 -------------------Combine and Sort
    # -------------------------------------------------------------
    ascending = METRIC_DIRECTION == "min"
    df_final = pd.concat([base_row, df_tuned], ignore_index=True)
    df_final = df_final.sort_values(by=USER_CUSTOM_METRIC, ascending=ascending)
    df_final.reset_index(drop=True, inplace=True)

    print(f"\nXXXXXXXXXXXXX Final Comparison Created | Shape: {df_final.shape}")
    print(f"xxxxxxxxxxxxxxx Best {USER_CUSTOM_METRIC}: {df_final.iloc[0][USER_CUSTOM_METRIC]:.6f}")
    print("=" * 80)

    return df_final

# Configuration

## Configuration and project validation

This section sets up the environment, defines tuning parameters, and validates the project type before proceeding with model optimization. Set your parameters here.

### DataRobot credentials

- `DR_ENDPOINT`: The API endpoint for your DataRobot instance.  
  Example: `'https://app.datarobot.com/api/v2'`
- `DR_API_TOKEN`: Your personal API token for authenticating with DataRobot.  
  Keep this secret and secure.

### Project and data setup

- `PROJECT_ID`: The unique identifier of your DataRobot project.  
  Example: `'692d8ac9f3a7f8fff356745f'`
- `FALLBACK_TRAINING_PATH`: Local CSV file path to load training data if the API-based fetch fails.  
  Example: `'Regression_Data.csv'`

### Model selection and sorting

- `TOP_N_MODELS`: Number of top leaderboard models to select for tuning.  
  Example: `5`
- `PREFERRED_METRICS`: Default metrics used to sort models before tuning.  
  For binary: `"LogLoss"` or `"AUC"`  
  For regression: `"RMSE"`, `"MAE"`, or `"R2"`
- `SORT_METRIC`: Optional override to sort models by a specific metric.  
  Set to `None` to use defaults from `PREFERRED_METRICS`.

### Custom metric configuration

- `USER_CUSTOM_METRIC`: Name of the custom metric used for optimization.  
  Example: `'clv_asymmetry_loss'`, `'my_cost_score'`
- `METRIC_FUNCTION`: The actual Python function that computes the custom metric.  
  Must be callable, e.g., `clv_asymmetry_loss`
- `METRIC_DIRECTION`: Whether to minimize or maximize the metric.  
  `"min"` for loss functions (e.g., RMSE, LogLoss)  
  `"max"` for gain metrics (e.g., AUC, R2)
- `METRIC_INPUT_TYPE`: Format of model predictions expected by the metric.  
  `"label"`: Binary class labels (0 or 1)  
  `"proba"`: Predicted probabilities for the positive class  
  `"numeric"`: Continuous values (for regression)

### Project validation

- Connects to the DataRobot project using `PROJECT_ID`.
- Detects the project type using `detect_project_type()`.
- Ensures the project is either:
  - `"binary"` for binary classification
  - `"regression"` for regression tasks

All configurations must be correctly set before proceeding. The notebook will raise errors if any required setting is missing or invalid.

In [55]:
# ----- DataRobot Credentials:
DR_ENDPOINT = "https://app.datarobot.com/api/v2"
DR_API_TOKEN = (
    "NjkyZmJlOWNmMjgwNTdmMjI5ZGEyN2ZmOmJGZWVXclhTT2dPQytIa0oyd09aR2ZaWWlUak5DUTYraXhjTHd3UTJWKzA9"
)

## ---------- Project Id :
PROJECT_ID = "692fd31ca5a8b5da9689a2a1"

dr.Client(endpoint=DR_ENDPOINT, token=DR_API_TOKEN)  ## End point and token-----


# ----- Fallback Data Path (for API-created projects) ----- pass the training dataset path :
# FALLBACK_TRAINING_PATH = "DR_Demo_AML_Alert.csv"
FALLBACK_TRAINING_PATH = "Regression_Data.csv"


# --------- How many top models to optimize
TOP_N_MODELS = 3

# ---------- Metric to sort models by (default: LogLoss for binary classification)
PREFERRED_METRICS = {
    "binary": "LogLoss",  # alternatives: "LogLoss" (lower better) or "AUC" (higher better)
    "regression": "RMSE",  # alternatives: "MAE", "R2" (R2 higher is better)
}


SORT_METRIC = None  #  # sort your models based on metric

# ------------HyperOpt budget: how many evaluations to try (increase for better tuning)
MAX_EVALS = 2  # 3,4,5

# USER_CUSTOM_METRIC = "my_cost_score"  # my_cost_score     # clv_asymmetry_loss
USER_CUSTOM_METRIC = "clv_asymmetry_loss"

# ---------------------
# Metric direction map (minimize or maximize)
# Use lower=better ("min") or higher=better ("max")
# Add new metrics here as needed.
# ---------------------
METRIC_FUNCTION = clv_asymmetry_loss  # actual python function i.e  (CUSTOMN METRICS)
METRIC_DIRECTION = "min"  # or "max"
METRIC_INPUT_TYPE = "numeric"  # or "label" or "numeric"  , proba   # Regression - numeric


##      ***********************************************************************************************************************************************************

######  ********************************************   END OF Configuration  ******************************************

if USER_CUSTOM_METRIC is None:
    raise RuntimeError(
        "!!!!!!!!111 USER_CUSTOM_METRIC must be defined. Example:\n"
        "    USER_CUSTOM_METRIC = 'my_cost_score'"
    )

if METRIC_FUNCTION is None or not callable(METRIC_FUNCTION):
    raise RuntimeError(
        "!!!!!!!!!!!11 METRIC_FUNCTION is missing or not callable.\n"
        "Example:\n"
        "    METRIC_FUNCTION = my_cost_score"
    )

if METRIC_DIRECTION not in ["min", "max"]:
    raise ValueError("!!!!!!!!!!1111 METRIC_DIRECTION must be either 'min' or 'max'.")

if METRIC_INPUT_TYPE not in ["label", "proba", "numeric"]:
    raise ValueError("!!!!!!!111 METRIC_INPUT_TYPE must be 'label', 'proba', or 'numeric'.")

print("\n===== CUSTOM METRIC CONFIGURATION =====")
print(f" Metric Function : {METRIC_FUNCTION.__name__}")
print(f" Direction       : {METRIC_DIRECTION}")
print(f" Input Type      : {METRIC_INPUT_TYPE}")
print("========================================\n")


# ======================================
#   ********************8  PROJECT VALIDATION (ONLY 2 TYPES)
# ======================================

print("=" * 80)
print(" SELECTING MODELS FOR OPTIMIZATION")
print("=" * 80 + "\n")

proj = dr.Project.get(PROJECT_ID)
print(f"→ Project ID: {PROJECT_ID}")

project_type = detect_project_type(proj)
print(f"→ Detected project type: {project_type}")


#  multiclass not allowed now
if project_type not in {"binary", "regression"}:
    raise RuntimeError(
        f"............... Unsupported project type: {project_type}\n"
        "This notebook now supports **only binary classification and regression**.\n"
        "For multiclass, build a separate notebook."
    )

print("\n✓ Project is valid for tuning.\n")


===== CUSTOM METRIC CONFIGURATION =====
 Metric Function : clv_asymmetry_loss
 Direction       : min
 Input Type      : numeric

 SELECTING MODELS FOR OPTIMIZATION

→ Project ID: 692fd31ca5a8b5da9689a2a1
→ Detected project type: regression

✓ Project is valid for tuning.



The next 3 cells will:

1. Select the top N models from your leaderboard
2. Loop through top N models and output a table of their tunable hyperparameters
3. Manually define a search space with 1. list of hyperparameters you would like to tune, and 2. their search spaces

The search space can be customized: use the table output `filtered_df` and accordingly update the search space.

# Select the top N models from the project Leaderboard

## Model selection for optimization

This section identifies and prepares the top-performing models from the DataRobot leaderboard for hyperparameter tuning.

### Step-by-step breakdown

1. **Project initialization**  
   Loads the project using the `PROJECT_ID` defined earlier. Prints the project ID for confirmation.

2. **Project type detection**  
   Uses `detect_project_type()` to determine if the project is:
   - `"binary"` for binary classification
   - `"regression"` for regression
   This helps in selecting the appropriate metric for sorting models.

3. **Sorting metric selection**  
   If `SORT_METRIC` is not explicitly set, it defaults to:
   - `"LogLoss"` for binary classification
   - `"RMSE"` for regression
   This metric is used to rank models for tuning.

4. **Recommended model retrieval**  
   Attempts to fetch the recommended model for deployment using `ModelRecommendation`. If found, its ID is stored to exclude it from tuning (to avoid overwriting production-ready models). This avoids modifying models that are ready for deployment.

5. **Leaderboard model fetching**  
   Retrieves all models from the project leaderboard using `proj.get_models()`.

6. **Top-N model selection**  
   Filters out the recommended model (if any). Selects the top `TOP_N_MODELS` from the remaining list for tuning.

7. **Model metadata extraction**  
   For each selected model:
   - Retrieves full model details
   - Extracts:
     - Model ID
     - Model Name
     - Validation and Holdout scores for the selected `SORT_METRIC`
   - Stores this information in a DataFrame.

8. **Sorting models by metric**  
   Sorts the DataFrame by the selected validation metric (e.g., `Validation LogLoss`). If the metric is missing, sorting is skipped with a warning.

9. **Final output**  
   Displays the top models selected for tuning.

### Why this matters

This step ensures that only the most promising models are passed to the tuning phase, improving efficiency and focusing optimization on high-potential candidates. It also avoids modifying any model already recommended for deployment.


In [ ]:
print("\n" + "=" * 80)
print(" SELECTING MODELS FOR OPTIMIZATION")
print("=" * 80 + "\n")

# ---------- Define the project
project_id = PROJECT_ID  # from configuration cell
print(" ***************  >>>>    Project Id is::", project_id)
proj = dr.Project.get(project_id)

# --------------------------------------------------------------
# ----------- Detect Project Type (for auto metric selection)
# --------------------------------------------------------------
project_type = detect_project_type(proj)
print(f" Detected project type: {project_type}")

# --------------------------------------------------------------
# --------- Dynamically choose sorting metric
# --------------------------------------------------------------
if SORT_METRIC is None:
    if project_type == "binary":
        SORT_METRIC = PREFERRED_METRICS["binary"]

    else:
        SORT_METRIC = PREFERRED_METRICS["regression"]

print(f"Sorting models by: {SORT_METRIC}\n")

# --------------------------------------------------------------
# ------------Get recommended deployment model (if any)
# --------------------------------------------------------------
try:
    recom_model = dr.ModelRecommendation.get(
        proj.id, dr.enums.RECOMMENDED_MODEL_TYPE.RECOMMENDED_FOR_DEPLOYMENT
    ).get_model()
    recom_model_id = recom_model.id
    print(f" Recommended model for deployment: {recom_model.model_type} | ID: {recom_model_id}")
except Exception as e:
    recom_model_id = None
    print("No recommended model found or error occurred:", e)

# --------------------------------------------------------------
# Fetch leaderboard models
# --------------------------------------------------------------
all_models = proj.get_models()
print(f" Total models retrieved: {len(all_models)}")

# --------------------------------------------------------------
# Filter: top N models (excluding recommended)
# --------------------------------------------------------------
top_models = [m for m in all_models if m.id != recom_model_id][:TOP_N_MODELS]
print(f" Selected top {len(top_models)} models for tuning.\n")

# --------------------------------------------------------------
# Extract relevant details for DataFrame
# --------------------------------------------------------------
model_data = []
for m in top_models:
    try:
        full_model = dr.Model.get(project_id, m.id)
        model_data.append(
            {
                "Model ID": full_model.id,
                "Model Name": full_model.model_type,
                f"Validation {SORT_METRIC}": full_model.metrics.get(SORT_METRIC, {}).get(
                    "validation"
                ),
                f"Holdout {SORT_METRIC}": full_model.metrics.get(SORT_METRIC, {}).get("holdout"),
            }
        )
    except Exception as e:
        print(f" >>>>>> Skipping {m.id} ({m.model_type}): {e}")
        continue

df_models = pd.DataFrame(model_data)

# --------------------------------------------------------------
# Sort by chosen validation metric (if available)
# --------------------------------------------------------------
sort_col = f"Validation {SORT_METRIC}"
if sort_col in df_models.columns:
    df_models = df_models.sort_values(by=sort_col, ascending=True)
    print(f">>> Sorted models by: {sort_col}")
else:
    print(f">>>>> Sort metric '{SORT_METRIC}' not found in model metrics — skipping sort.")

print(f"\n!!!!!!!!!11 Final model list shape: {df_models.shape}")
display(df_models.head())

if df_models.empty:
    print("\n!!!!!!!!!!!!! No successful tuning results found or no models available.")
else:
    print("\n>>>>>>>>>>>>> Models successfully selected for optimization.")


 SELECTING MODELS FOR OPTIMIZATION

 ***************  >>>>    Project Id is:: 692fd31ca5a8b5da9689a2a1
 Detected project type: regression
🔹 Sorting models by: RMSE

 Recommended model for deployment: Light Gradient Boosting on ElasticNet Predictions  | ID: 692fd4f4d120ebaebc84bdd8
 Total models retrieved: 9
 Selected top 3 models for tuning.

>>> Sorted models by: Validation RMSE

!!!!!!!!!11 Final model list shape: (3, 4)


,Model ID,Model Name,Validation RMSE,Holdout RMSE
0,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,124.87085,133.21948
1,692fd37102faa1b369c44afa,Light Gradient Boosting on ElasticNet Predicti...,126.70173,121.59392
2,692fd37102faa1b369c44af6,Ridge Regressor,138.34444,132.64639



>>>>>>>>>>>>> Models successfully selected for optimization.


## Fetch tunable parameters for top models

This section extracts tunable hyperparameters from the top selected models to prepare for optimization.

### Step-by-step breakdown

1. **Initialization**  
   Prints a header to indicate the start of parameter extraction. Detects the project type (`binary` or `regression`) for logging and consistency.

2. **Loop through selected models**  
   Iterates over each model in `df_models` (previously selected top models). For each model:
   - Retrieves the full model object using its ID.
   - Calls `get_current_hyperparams()` to extract tunable parameters.
   - If parameters are found:
     - Adds metadata like Model ID, Model Name, and Project Type.

3. **Combine all parameters**  
   Concatenates all individual parameter DataFrames into a single `df_all_params`.

4. **Filter parameters by task name**  
   Adds lowercase helper columns for `task_name` and `model_name` to enable case-insensitive filtering. Filters rows where the model name is found within the task name (to isolate relevant tuning steps).

5. **Final output**  
   Prints the shape of the final filtered DataFrame. Displays the filtered parameter set (`filtered_df`) which will be used for tuning.

### Why this matters

This step ensures that only the relevant and tunable parameters for each model are extracted and prepared for optimization. Filtering by task name ensures that parameters are correctly aligned with the model's core algorithm step (e.g., classifier or regressor).

In [57]:
print("\n" + "=" * 80)
print(" FETCHING TUNABLE PARAMETERS FOR TOP MODELS")
print("=" * 80 + "\n")

all_param_data = []

# Detect once (for logging / awareness)
project_type = detect_project_type(proj)
print(f"Detected project type: {project_type}")

for _, row in df_models.iterrows():
    model_id = row["Model ID"]
    model_name = row["Model Name"]

    print(f"\n>>>>>>>>>>>>>>>> Fetching tunable parameters for {model_name} ({model_id})...")
    try:
        model = dr.Model.get(proj.id, model_id)
        params_df = get_current_hyperparams(model)

        if not params_df.empty:
            # ---------------Add model metadata
            params_df["Model ID"] = model_id
            params_df["Model Name"] = model_name
            params_df["Project Type"] = project_type
            all_param_data.append(params_df)
        else:
            print(f" !!!!!!!!!!!111 No tunable parameters found for {model_name} ({model_id}).")

    except Exception as e:
        print(f" !!!!!!!!!!!!!!!! Skipping model {model_id} ({model_name}): {e}")
        continue

# ----------Combining all parameter data into one dataframe

if all_param_data:
    df_all_params = pd.concat(all_param_data, ignore_index=True)
else:
    df_all_params = pd.DataFrame(
        columns=[
            "Model ID",
            "Model Name",
            "Project Type",
            "task_name",
            "parameter_name",
            "parameter_name_type",
            "current_value",
            "default_value",
            "param_type",
            "min",
            "max",
        ]
    )

print(
    f"\n Final dataframe created with {df_all_params.shape[0]} rows and {df_all_params.shape[1]} columns."
)

# ---------- Helper lowercase columns for filtering
df_all_params["task_name_lower"] = df_all_params["task_name"].str.lower()
df_all_params["model_name_lower"] = df_all_params["Model Name"].str.lower()

# ----------Apply contains-based filter (same logic)
filtered_df = df_all_params[
    df_all_params.apply(lambda x: x["model_name_lower"] in x["task_name_lower"], axis=1)
].copy()

# ----------Drop helper columns
filtered_df.drop(columns=["task_name_lower", "model_name_lower"], inplace=True, errors="ignore")

print(
    f" Filtered dataframe created with {filtered_df.shape[0]} rows (matching task_name ↔ model_name)."
)
print(filtered_df.shape)
display(filtered_df)


 FETCHING TUNABLE PARAMETERS FOR TOP MODELS

Detected project type: regression

>>>>>>>>>>>>>>>> Fetching tunable parameters for Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss) (692fd37102faa1b369c44af7)...

>>>>>>>>>>>>>>>> Fetching tunable parameters for Light Gradient Boosting on ElasticNet Predictions  (692fd37102faa1b369c44afa)...

>>>>>>>>>>>>>>>> Fetching tunable parameters for Ridge Regressor (692fd37102faa1b369c44af6)...

 Final dataframe created with 58 rows and 11 columns.
 Filtered dataframe created with 42 rows (matching task_name ↔ model_name).
(42, 11)


,task_name,parameter_name,parameter_name_type,current_value,default_value,param_type,min,max,Model ID,Model Name,Project Type
0,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,enet_alpha,enet_alpha_select,0.5,0.5,select,NaN,NaN,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,regression
1,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,enet_lambda,enet_lambda_select,0.003474,0.003474,select,NaN,NaN,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,regression
2,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,fit_alpha_scaler,fit_alpha_scaler_select,True,True,select,NaN,NaN,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,regression
3,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,fit_intercept,fit_intercept_select,True,True,select,NaN,NaN,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,regression
4,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,fit_tweedie_p,fit_tweedie_p_select,False,False,select,NaN,NaN,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,regression
5,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,loss,loss_select,squared,squared,select,NaN,NaN,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,regression
6,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,max_iter,max_iter_int,100,100,int,1.000000e+00,1.000000e+06,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,regression
7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,random_state,random_state_int,1234,1234,int,0.000000e+00,1.000000e+09,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,regression
8,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,tol,tol_float,0.0001,0.0001,float,1.000000e-10,1.000000e+10,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,regression
9,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,tweedie_p,tweedie_p_float,1.5,1.5,float,1.000000e+00,2.000000e+00,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Leas...,regression


In [58]:
## optional

## Saving the
filtered_df.to_csv(
    "/home/notebooks/storage/Regression_search_space/Regression_search_space_2.csv", index=False
)

## Custom search space definitions for model tuning

This section defines the hyperparameter search spaces for various classification and regression models supported in DataRobot. These search spaces are used by the HyperOpt tuner to explore optimal hyperparameter combinations during model optimization.

### What is `SEARCH_SPACES`?

- `SEARCH_SPACES` is a dictionary where:
  - **Keys** = Exact model names as they appear in DataRobot (e.g., `"RandomForest Classifier (Gini)"`)
  - **Values** = Hyperparameter search definitions using `hyperopt` functions like:
    - `hp.uniform()` for continuous ranges
    - `hp.quniform()` for discrete ranges
    - `hp.choice()` for categorical options
    - `hp.loguniform()` for log-scaled values

### Saving the filtered search space

After extracting tunable parameters from the top models, the filtered DataFrame (`filtered_df`) is saved to disk for reference or reuse. Go through the file `filtered_df` and understand the hyperparameters and update the search space with model name and its hyperparameters.

In [59]:
# start modifying the search space overe here:

SEARCH_SPACES = {
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Random Forest:
    "RandomForest Classifier (Gini)": {
        "n_estimators": hp.quniform("n_estimators", 100, 800, 50),
        "min_samples_split": hp.quniform("min_samples_split", 2, 20, 1),
        "min_samples_leaf": hp.quniform("min_samples_leaf", 1, 15, 1),
    },
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> eXtreme Gradient Boosted Trees:
    "eXtreme Gradient Boosted Trees Classifier with Early Stopping": {
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.3),
        "min_child_weight": hp.quniform("min_child_weight", 1, 6, 1),
        "subsample": hp.uniform("subsample", 0.6, 1.0),
        "colsample_bytree": hp.uniform("colsample_bytree", 0.6, 1.0),
        "n_estimators": hp.quniform("n_estimators", 100, 800, 50),
    },
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> RuleFit Classifier:
    "RuleFit Classifier": {
        "max_depth": hp.quniform("max_depth", 2, 10, 1),
        "n_estimators": hp.quniform("n_estimators", 100, 800, 50),
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.3),
        "min_samples_leaf": hp.quniform("min_samples_leaf", 1, 5, 1),
    },
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Generalized Additive2 Model:
    "Generalized Additive2 Model": {
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.3),
        "max_depth": hp.quniform("max_depth", 2, 10, 1),
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
        "n_estimators": hp.quniform("n_estimators", 100, 800, 50),
        "min_child_weight": hp.quniform("min_child_weight", 1, 6, 1),
    },
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Light Gradient Boosted Trees:
    "Light Gradient Boosted Trees Classifier with Early Stopping": {
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.3),
        "num_leaves": hp.quniform("num_leaves", 20, 120, 10),
        "max_depth": hp.quniform("max_depth", 3, 10, 1),
        "min_child_samples": hp.quniform("min_child_samples", 5, 50, 5),
        "min_child_weight": hp.quniform("min_child_weight", 1, 6, 1),
        "n_estimators": hp.quniform("n_estimators", 100, 800, 50),
    },
    ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Elastic Net:
    # "ElasticNet": {
    #   "enet_alpha": hp.loguniform("enet_alpha", -9.2, 0.0),
    #  "enet_lambda": hp.uniform("enet_lambda", 0.0, 1.0)
    # },
    ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>  RandomForest Classifier (Entropy):
    "RandomForest Classifier (Entropy)": {
        "n_estimators": hp.quniform("n_estimators", 100, 800, 50),
        "min_samples_split": hp.quniform("min_samples_split", 2, 20, 1),
        "min_samples_leaf": hp.quniform("min_samples_leaf", 1, 15, 1),
    },
    ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>   Keras:
    "Keras": {
        "learning_rate": hp.loguniform("learning_rate", -9.2, -2.3),
        "batch_size": hp.choice("batch_size", [16, 32, 64, 128]),
        "activation": hp.choice("activation", ["relu", "elu", "selu"]),
    },
    ## >>>>>>>>>>>>>>>>>>>>>>>>>>>>> GAM:
    "GAM": {
        "learning_rate": hp.loguniform("learning_rate", -6.9, -2.3),  # LOG-SCALE: 0.001 to 0.1
        "max_bins": hp.quniform("max_bins", 128, 512, 64),
    },
    ## >>>>>>>>>>>>>>>>>>>>>> Keras Slim Residual:
    "Keras Slim Residual": {
        "learning_rate": hp.loguniform("learning_rate", -9.2, -2.3),  # LOG-SCALE: 0.0001 to 0.1
        "batch_size": hp.choice("batch_size", [16, 32, 64, 128]),
        "epoch": hp.quniform("num_residual_blocks", 2, 100, 1),
    },
    ##  ##########################################   Regression Model :
    ## Elastic-Net Regressor :
    "Elastic-Net Regressor": {
        "enet_alpha": hp.loguniform("enet_alpha", -9.2, 0.0),
        "enet_lambda": hp.uniform("enet_lambda", 0.0, 1.0),
    },
    "Light Gradient Boosting on ElasticNet Predictions": {
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.3),
        "num_leaves": hp.quniform("num_leaves", 20, 120, 10),
        "max_depth": hp.quniform("max_depth", 3, 10, 1),
        "n_estimators": hp.quniform("n_estimators", 100, 800, 50),
    },
    "eXtreme Gradient Boosted Trees Regressor": {
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.3),
        "min_child_weight": hp.quniform("min_child_weight", 1, 6, 1),
        "subsample": hp.uniform("subsample", 0.6, 1.0),
        "colsample_bytree": hp.uniform("colsample_bytree", 0.6, 1.0),
        "n_estimators": hp.quniform("n_estimators", 100, 800, 50),
    },
    "Ridge Regressor": {"tweedie_p": hp.quniform("tweedie_p", 1.0, 2.0, 0.1)},
    "RuleFit Regressor": {
        "max_depth": hp.quniform("max_depth", 2, 10, 1),
        "n_estimators": hp.quniform("n_estimators", 100, 800, 50),
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.3),
        "min_samples_leaf": hp.quniform("min_samples_leaf", 1, 5, 1),
    },
    "Light Gradient Boosted Trees Regressor with Early Stopping": {
        "num_leaves": hp.quniform("num_leaves", 2, 1000, 50),
        "n_estimators": hp.quniform("n_estimators", 100, 800, 50),
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.3),
    },
}

# Print top N models to be tuned, and their available search spaces

In [ ]:
print("\n" + "=" * 80)
print("Allowed search spaces for top X models")
print("=" * 80 + "\n")
for _, row in df_models.iterrows():
    model_name = row["Model Name"]
    print("\n---------------------------------------")
    print(f" Model: {model_name}")
    space = get_search_space(model_name)
    print(f" Search Space Keys: {list(space.keys())}")


Allowed Search Spaces for Top X Models


---------------------------------------
 Model: Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss)
 Search space found for model: Elastic-Net Regressor
 Search Space Keys: ['enet_alpha', 'enet_lambda']

---------------------------------------
 Model: Light Gradient Boosting on ElasticNet Predictions 
 Search space found for model: Light Gradient Boosting on ElasticNet Predictions
 Search Space Keys: ['learning_rate', 'num_leaves', 'max_depth', 'n_estimators']

---------------------------------------
 Model: Ridge Regressor
 Search space found for model: Ridge Regressor
 Search Space Keys: ['tweedie_p']


# Get training data

## `get_training_data()`

This function retrieves the training dataset for a given DataRobot project, supporting both UI-created and API-created projects across classification and regression types.

---

###  Function purpose

- Attempts to load the training dataset directly from the DataRobot backend.
- If the dataset cannot be fetched (e.g., for API-created projects), it falls back to a local CSV file.
- Ensures the target column exists in the dataset and logs helpful messages throughout.

---

---

###  Example Use Case

This function is especially useful when:
- Projects are created via the API and don’t have a linked dataset.
- You want to ensure robustness in notebooks that may run across different environments.


In [61]:
def get_training_data(proj, fallback_path=None):
    """
    Safely fetch project training data as a pandas DataFrame.
    Works for both UI-created and API-created projects.
    Supports both regression and classification.
    """

    df = None
    project_type = detect_project_type(proj)
    target_name = proj.target

    print(f"\n{'='*80}")
    print(f" FETCHING TRAINING DATA  (Project Type: {project_type})")
    print(f"{'='*80}")

    # ----------------- Try pulling directly from DataRobot -----------------
    try:
        dataset_func = getattr(proj, "get_dataset", None)
        if callable(dataset_func):
            dataset_obj = dataset_func()
            if dataset_obj is not None:
                df = dataset_obj.get_as_dataframe()
                print(f">> Loaded dataset from DataRobot backend. Shape: {df.shape}")

                if target_name in df.columns:
                    print(f">>>> Target column '{target_name}' found in dataset.")
                else:
                    print(f">>>>>> Target column '{target_name}' not found in dataset!")

                return df
    except Exception as e:
        print(f">>>>>>>>> Could not fetch dataset from DataRobot backend: {e}")

    # ----------------- Fallback to local CSV -----------------
    if fallback_path:
        if os.path.exists(fallback_path):
            try:
                df = pd.read_csv(fallback_path)
                print(
                    f">>>>>>> Loaded fallback dataset from local file: {fallback_path} | Shape: {df.shape}"
                )

                if target_name in df.columns:
                    print(f">>>>>>>>>>> Target column '{target_name}' found in fallback dataset.")
                else:
                    print(
                        f">>>>>>>>>>>>>>>>>>>>>>>>> Target column '{target_name}' not found in fallback dataset!"
                    )

                return df
            except Exception as e:
                raise RuntimeError(f"❌ Failed to load fallback CSV ({fallback_path}): {e}")
        else:
            print(f"!!!!!!!!!!!!!!!!! Fallback path not found: {fallback_path}")

    # ----------------- If all fails -----------------
    raise RuntimeError(
        "❌ Could not load dataset from DataRobot or fallback path.\n"
        "If project was created via API (Project.create), provide fallback_path to local CSV."
    )

# Hyperparameter optimization

## Model optimization loop

This section performs the core optimization process by tuning each selected model using the defined custom metric and search space.

### Step-by-step breakdown

#### 1. Project type validation
- Detects the project type using `detect_project_type(proj)`.
- Ensures the project is either:
  - `"binary"` for binary classification
  - `"regression"` for regression

#### 2. Model tuning loop
For each model in the `df_models` DataFrame:
- Logs the model name and ID.
- Retrieves the appropriate hyperparameter `search_space` using `get_search_space(model_name)`.
- Records the start time for runtime tracking.
- Calls `tune_and_compare_model()` to:
  - Run HyperOpt tuning using the custom metric
  - Evaluate performance improvements
  - Return a DataFrame of tuning results

#### 3. Combine and summarize results
- After all models are processed:
  - Calculates total runtime for the entire optimization loop.
  - Logs the total time taken.
  - The `all_results` list now contains performance comparisons for each tuned model.

### Why this matters

This loop automates the process of:
- Selecting models
- Applying custom metric-driven tuning
- Tracking performance improvements
- Collecting results for final analysis

It ensures consistent, scalable, and efficient optimization across multiple models in a DataRobot project.

In [62]:
print("\n" + "=" * 80)
print(" STARTING OPTIMIZATION LOOP")
print("=" * 80 + "\n")

start_time = time.time()
all_results = []

# --------------------------------------------------------------
# 1. Detect project type (binary or regression only)
# --------------------------------------------------------------
project_type = detect_project_type(proj)
print(f"Detected project type: {project_type}")

if project_type not in {"binary", "regression"}:
    raise RuntimeError(
        f"XXXXX  Unsupported project type '{project_type}'. "
        "This notebook only supports binary classification & regression."
    )


print(f" >>>>>>>>>>> Optimization will use custom metric: {METRIC_FUNCTION.__name__}")
print("=" * 80)

# --------------------------------------------------------------
# 2. Tuning Loop
# --------------------------------------------------------------
for idx, row in df_models.iterrows():
    model_id = row["Model ID"]
    model_name = row["Model Name"]

    print("\n" + "=" * 120)
    print(f" [{idx+1}/{len(df_models)}] Tuning model: {model_name} | ID: {model_id}")
    print("=" * 120)

    search_space = get_search_space(model_name)
    model_start = time.time()

    try:
        # ---- Run tuning ----
        df_result = tune_and_compare_model(
            proj=proj,
            base_model_id=model_id,
            search_space=search_space,
            max_evals=MAX_EVALS,
            fallback_path=FALLBACK_TRAINING_PATH,
        )

        # ---- Add metadata ----
        df_result["Model ID"] = model_id
        df_result["Model Name"] = model_name
        df_result["Project Type"] = project_type
        df_result["Metric Used"] = USER_CUSTOM_METRIC
        df_result["Runtime (min)"] = round((time.time() - model_start) / 60, 2)

        all_results.append(df_result)

        print(
            f"✔ Completed tuning for {model_name} "
            f"in {df_result['Runtime (min)'].iloc[0]} minutes"
        )

    except Exception as e:
        print(f"XXXX Skipping {model_name} due to error: {e}")
        continue


# --------------------------------------------------------------
# 3. Combine all results
# --------------------------------------------------------------
end_time = time.time()
total_time = round((end_time - start_time) / 60, 2)

print("\n" + "=" * 80)
print(f" COMPLETED ENTIRE OPTIMIZATION IN {total_time} MINUTES")
print("=" * 80 + "\n")


 STARTING OPTIMIZATION LOOP

Detected project type: regression
 >>>>>>>>>>> Optimization will use custom metric: clv_asymmetry_loss

 [1/3] Tuning model: Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss) | ID: 692fd37102faa1b369c44af7
 Search space found for model: Elastic-Net Regressor

 FETCHING TRAINING DATA  (Project Type: regression)
>> Loaded dataset from DataRobot backend. Shape: (850, 96)
>>>> Target column 'CLV_2' found in dataset.
1. Training data shape: (850, 96)
2. Target variable: CLV_2
3. Project Type: regression
XX Starting tuning for: Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss) | ID: 692fd37102faa1b369c44af7
XXX Custom Metric: clv_asymmetry_loss
XXXX  Metric Function: clv_asymmetry_loss
XXXXX  Direction: min
X Base clv_asymmetry_loss: 187.571906
*********** Trying params: {'enet_alpha': 0.21140153624993946, 'enet_lambda': 0.09839430298593588}
XXXXXXXXXXXXXX Tuned clv_asymmetry_loss: 280.625171 | Δ = -93.053265
*********** Trying params: {'

# Result analysis

In [63]:
# --------------------------------------------------------------
#  ----------FINAL AGGREGATION AND RESULT VISUALIZATION
# --------------------------------------------------------------

# ---- Combine all results ----
if all_results:
    df_all_results = pd.concat(all_results, ignore_index=True)
    print(f"\n Combined tuning results created successfully. Shape: {df_all_results.shape}")
else:
    df_all_results = pd.DataFrame()
    print(">>>>>>>>>  No results collected — all models failed or were skipped.")

# --------------------------------------------------------------
# ------ Identify metric key and sorting direction dynamically
# --------------------------------------------------------------
metric_key = USER_CUSTOM_METRIC if "USER_CUSTOM_METRIC" in globals() else "metric"

if metric_key not in df_all_results.columns:
    metric_key = "metric"  # fallback
    print(f">>>>>>>>>>> Metric '{USER_CUSTOM_METRIC}' not found — using 'metric' column instead.")

direction = (
    METRIC_DIRECTION
    if isinstance(METRIC_DIRECTION, str)
    else METRIC_DIRECTION.get(metric_key, "min")
)
ascending = True if direction == "min" else False
direction_label = "lower=better" if ascending else "higher=better"

# ---- Sort results ----
df_all_results = df_all_results.sort_values(by=metric_key, ascending=ascending)
df_all_results.reset_index(drop=True, inplace=True)

print(f"\n  Results sorted by '{metric_key}' ({direction_label}).")

# --------------------------------------------------------------
# --------- Apply Heatmap Styling
# --------------------------------------------------------------

# Use red→green gradient for cost (or metric) column
styled_results = df_all_results.style.background_gradient(
    subset=[metric_key],
    cmap="RdYlGn_r" if ascending else "RdYlGn",  # green = better, red = worse
)

# Add gradient to delta columns if available
for delta_col in ["delta_cost", "delta_metric", f"delta_{metric_key}", "delta"]:
    if delta_col in df_all_results.columns:
        try:
            styled_results = styled_results.background_gradient(
                subset=[delta_col], cmap="YlOrBr", vmin=-0.1, vmax=0.1
            )
        except Exception as e:
            print(f"XXXX Could not apply gradient to {delta_col}: {e}")

# --------------------------------------------------------------
# >>>>>>>>>>>>> Display Column Definitions (Markdown)
# --------------------------------------------------------------
display(
    Markdown(
        f"""
###  Results Column Definitions

| Column | Description |
|---------|-------------|
| **trial_number** | Sequential number of the tuning trial (0 = baseline/original model). |
| **model_id** | DataRobot model identifier. |
| **model_name** | Model type (e.g., RandomForest, XGBoost). |
| **type** | Indicates whether the row is Original (baseline) or Tuned. |
| **{metric_key}** | Final optimization metric value ({direction_label}). |
| **delta_{metric_key}** | Difference between baseline and tuned model metric.<br>🟢 Positive → improvement (tuned model better).<br>🔴 Negative or zero → no gain or degradation. |
| **parameters** | Model tuning hyperparameters used in that trial. |
"""
    )
)

# --------------------------------------------------------------
# ----------- Display Styled Results
# --------------------------------------------------------------
print("\n")
print("***************************** RESULTS SUMMARY *****************************\n")
display(styled_results)

# --------------------------------------------------------------
# ------------ Identify and Display Best Model
# --------------------------------------------------------------
if not df_all_results.empty:
    best_model = (
        df_all_results.loc[df_all_results[metric_key].idxmin()]
        if ascending
        else df_all_results.loc[df_all_results[metric_key].idxmax()]
    )

    print("\n------ ********************** BEST MODEL (Based on Metric) **********************\n")
    print(best_model)
    print("\n==========================================================================\n")

# --------------------------------------------------------------
# ------------- Display Total Time
# --------------------------------------------------------------
print(f"\n Total time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
print("***************************** OPTIMIZATION COMPLETE *****************************")


 Combined tuning results created successfully. Shape: (9, 19)

  Results sorted by 'clv_asymmetry_loss' (lower=better).



###  Results Column Definitions

| Column | Description |
|---------|-------------|
| **trial_number** | Sequential number of the tuning trial (0 = baseline/original model). |
| **model_id** | DataRobot model identifier. |
| **model_name** | Model type (e.g., RandomForest, XGBoost). |
| **type** | Indicates whether the row is Original (baseline) or Tuned. |
| **clv_asymmetry_loss** | Final optimization metric value (lower=better). |
| **delta_clv_asymmetry_loss** | Difference between baseline and tuned model metric.<br>🟢 Positive → improvement (tuned model better).<br>🔴 Negative or zero → no gain or degradation. |
| **parameters** | Model tuning hyperparameters used in that trial. |




***************************** RESULTS SUMMARY *****************************



,trial,model_id,model_name,type,clv_asymmetry_loss,delta,status,param_enet_alpha,param_enet_lambda,Model ID,Model Name,Project Type,Metric Used,Runtime (min),param_learning_rate,param_max_depth,param_n_estimators,param_num_leaves,param_tweedie_p
0,0,692fd37102faa1b369c44afa,Light Gradient Boosting on ElasticNet Predictions,Original,168.560443,0.000000,ok,nan,nan,692fd37102faa1b369c44afa,Light Gradient Boosting on ElasticNet Predictions,regression,clv_asymmetry_loss,3.100000,nan,nan,nan,nan,nan
1,2,692fd952ad0f5df96189a0b3,Light Gradient Boosting on ElasticNet Predictions,Tuned,169.809753,-1.249310,ok,nan,nan,692fd37102faa1b369c44afa,Light Gradient Boosting on ElasticNet Predictions,regression,clv_asymmetry_loss,3.100000,0.049762,7.000000,650.000000,30.000000,nan
2,1,692fd8fe58440a870d56724c,Light Gradient Boosting on ElasticNet Predictions,Tuned,169.924455,-1.364012,ok,nan,nan,692fd37102faa1b369c44afa,Light Gradient Boosting on ElasticNet Predictions,regression,clv_asymmetry_loss,3.100000,0.032957,6.000000,250.000000,80.000000,nan
3,0,692fd37102faa1b369c44af6,Ridge Regressor,Original,186.539415,0.000000,ok,nan,nan,692fd37102faa1b369c44af6,Ridge Regressor,regression,clv_asymmetry_loss,3.140000,nan,nan,nan,nan,nan
4,1,692fd9bad89a0b04b789a1b8,Ridge Regressor,Tuned,186.539415,0.000000,ok,nan,nan,692fd37102faa1b369c44af6,Ridge Regressor,regression,clv_asymmetry_loss,3.140000,nan,nan,nan,nan,1.400000
5,2,692fda094a11aecc4189aecc,Ridge Regressor,Tuned,186.539415,0.000000,ok,nan,nan,692fd37102faa1b369c44af6,Ridge Regressor,regression,clv_asymmetry_loss,3.140000,nan,nan,nan,nan,1.600000
6,0,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss),Original,187.571906,0.000000,ok,nan,nan,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss),regression,clv_asymmetry_loss,3.040000,nan,nan,nan,nan,nan
7,1,692fd84aa56e28f6f8567ec8,Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss),Tuned,280.625171,-93.053265,ok,0.211402,0.098394,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss),regression,clv_asymmetry_loss,3.040000,nan,nan,nan,nan,nan
8,2,692fd898532032b903567405,Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss),Tuned,459.572638,-272.000732,ok,0.120250,0.471101,692fd37102faa1b369c44af7,Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss),regression,clv_asymmetry_loss,3.040000,nan,nan,nan,nan,nan



------ ********************** BEST MODEL (Based on Metric) **********************

trial                                                                  0
model_id                                        692fd37102faa1b369c44afa
model_name             Light Gradient Boosting on ElasticNet Predicti...
type                                                            Original
clv_asymmetry_loss                                            168.560443
delta                                                                0.0
status                                                                ok
param_enet_alpha                                                     NaN
param_enet_lambda                                                    NaN
Model ID                                        692fd37102faa1b369c44afa
Model Name             Light Gradient Boosting on ElasticNet Predicti...
Project Type                                                  regression
Metric Used                             

In [64]:
df_all_results.shape

(9, 19)

## Conclusion

In this notebook, you:

- Loaded the training dataset and detected project type automatically
- Defined business-driven custom metrics for both classification and regression
- Performed Bayesian hyperparameter optimization (HyperOpt)
- Compared tuned models against baseline using the custom metric
- Selected the best-performing models dynamically. The models will be reflected in the UI.
- Optionally predicted the scoring dataset with top 5 models (tuned/original). This section is below this cell.

This workflow ensures that models are aligned with business objectives rather than generic ML metrics. The same structure can be reused for:
- CLV prediction
- Fraud detection
- Churn modeling
- Insurance risk modeling
- Any metric-sensitive domain requiring asymmetric cost functions

By integrating custom loss functions with DataRobot tuning capabilities, you achieve a model optimization process that is transparent, repeatable, and business-driven.


# Optional

Prediction for scoring dataset

## Model scoring pipeline (regression + classification)

This section scores the top 5 tuned models on a new dataset and generates predictions for the scoring dataset so that you can compare the performance of the model with the actuals later.

### 1: Load scoring dataset

- Reads the scoring dataset from a local CSV file.
- Uploads the dataset to the DataRobot project using `proj.upload_dataset()`.
- Detects the project type (`binary` or `regression`) for downstream logic.

### 2: Select top 5 models

- Sorts the `df_all_results` DataFrame by the custom metric (`USER_CUSTOM_METRIC`).
- Uses the metric direction (`min` or `max`) to determine sorting order.
- Selects the top 5 models for scoring and displays their metadata.

### 3: Generate predictions

For each of the top 5 models:
- Loads the model using its ID.
- Requests predictions on the uploaded scoring dataset.
- Based on the project type:
  - **Binary classification**:
    - Extracts the probability column (e.g., `class_1`, `positive_probability`, or `prediction`).
    - Applies a threshold (default = 0.5) to convert probabilities into binary predictions.
    - Stores both probabilities and predicted labels.
  - **Regression**:
    - Extracts the `prediction` column directly.
- Adds prediction columns to the `predsAll` DataFrame with model-specific prefixes.

### 4: Create final output

- Builds a summary table (`scoring_summary_df`) mapping each model to its corresponding prediction column.
- Saves the full prediction dataset (`predsAll`) to the results folder.

In [ ]:
## Read scoring data

filename = "Regression_scoring_dataset_2.csv"
# filename= "DR_Demo_AML_Alert_SCORING.csv"
proj = dr.Project.get(project_id)
print(f"Project ID: {project_id}")
test = pd.read_csv(filename, encoding="ISO-8859-1")

print("\n" + "=" * 80)

print(f"Total shape of the scoring dataset: {test.shape}")

dataset = proj.upload_dataset(filename)

Project Id --- 692fd31ca5a8b5da9689a2a1

 Total shape of the Scoring dataset is - (136, 96)


In [68]:
# =====================================================================================================
# ------------ UNIVERSAL MODEL SCORING PIPELINE (Regression + Classification)
# =====================================================================================================

print("\n============================ SCORING PHASE STARTED ============================\n")

# --------------------------------------------------------------
# 1️) Load Scoring Dataset
# --------------------------------------------------------------
try:
    predsAll = test.copy()
    print(f"> Loaded scoring dataset successfully. Shape: {predsAll.shape}")
except Exception as e:
    raise RuntimeError(f"X Please load your scoring dataset as `test`. Error: {e}")

target_name = proj.target
if target_name in predsAll.columns:
    predsAll.rename(columns={target_name: "actuals"}, inplace=True)
else:
    predsAll["actuals"] = np.nan

project_type = detect_project_type(proj)
print(f"*** Detected Project Type: {project_type}")

# --------------------------------------------------------------
# 2) Select Top 5 Models Based on Custom Metric
# --------------------------------------------------------------
metric_key = USER_CUSTOM_METRIC if USER_CUSTOM_METRIC in df_all_results.columns else "metric"
direction = (
    METRIC_DIRECTION
    if isinstance(METRIC_DIRECTION, str)
    else METRIC_DIRECTION.get(metric_key, "min")
)
ascending = True if direction == "min" else False

df_top_models = (
    df_all_results.sort_values(by=metric_key, ascending=ascending).head(5).reset_index(drop=True)
)

print(
    f"\n>>>> Selected Top 5 Models based on {metric_key} ({'lower=better' if ascending else 'higher=better'})"
)
display(df_top_models[["model_id", "model_name", "type", metric_key]])

# --------------------------------------------------------------
# 3️) Generate Predictions for Each Model
# --------------------------------------------------------------
for idx, row in df_top_models.iterrows():
    model_id = row["model_id"]
    model_name = row.get("model_name", "Unknown")
    model_type = row.get("type", "Tuned")

    print(f"\n Predicting with Model {model_id} ({model_type})")
    model = dr.Model.get(proj.id, model_id)

    # ------------ Request predictions
    pred_job = model.request_predictions(dataset.id)
    preds = pred_job.get_result_when_complete()

    model_prefix = f"{model_id[-6:]}_{model_type}"

    # ----- Binary Classification -----
    if project_type == "binary":
        # Detect probability column
        prob_col = None
        for c in preds.columns:
            if c.lower() in ["class_1", "positive_probability", "prediction"]:
                prob_col = c
                break

        if not prob_col:
            raise KeyError(f"X No positive probability column found for model {model_id}")

        threshold = row.get("threshold", 0.5)

        # ----Save columns
        predsAll[f"{model_prefix}_prob"] = preds[prob_col]
        predsAll[f"{model_prefix}_pred"] = (preds[prob_col] >= threshold).astype(int)
        predsAll[f"{model_prefix}_threshold"] = threshold

    # ----- Regression -----
    elif project_type == "regression":
        if "prediction" not in preds.columns:
            raise KeyError(f"X Missing prediction column for model {model_id}")

        predsAll[f"{model_prefix}_pred"] = preds["prediction"]

    else:
        raise RuntimeError("X Unsupported project type for scoring.")

    print(f"> Predictions added for {model_prefix}")

# --------------------------------------------------------------
# 4️) Create Final Output (ONLY predictions + metadata)
# --------------------------------------------------------------

# -=-------------- BuildING structured results (one row per model)
scoring_summary = []

for idx, row in df_top_models.iterrows():
    model_id = row["model_id"]
    model_name = row.get("model_name", "Unknown")
    model_type = row.get("type", "Tuned")
    model_prefix = f"{model_id[-6:]}_{model_type}"

    scoring_summary.append(
        {
            "model_id": model_id,
            "model_name": model_name,
            "type": model_type,
            "prediction_column": f"{model_prefix}_pred",
        }
    )

scoring_summary_df = pd.DataFrame(scoring_summary)
print("\n MODEL–PREDICTION COLUMN MAP:")
display(scoring_summary_df)

# -----------Save scoring dataset
output_file = "/home/notebooks/storage/Results/Final_Scoring_Top5.csv"
predsAll.to_csv(output_file, index=False)

print(f"\n Final predictions saved to: {output_file}")
print("\n============================ SCORING PHASE COMPLETE ============================\n")

# Show prediction columns
pred_cols = [c for c in predsAll.columns if c.endswith("_pred")]
print("\n Prediction columns generated:\n")
print(pred_cols)


============================ SCORING PHASE STARTED ============================

> Loaded scoring dataset successfully. Shape: (136, 96)
*** Detected Project Type: regression

>>>> Selected Top 5 Models based on clv_asymmetry_loss (lower=better)


,model_id,model_name,type,clv_asymmetry_loss
0,692fd37102faa1b369c44afa,Light Gradient Boosting on ElasticNet Predicti...,Original,168.560443
1,692fd952ad0f5df96189a0b3,Light Gradient Boosting on ElasticNet Predicti...,Tuned,169.809753
2,692fd8fe58440a870d56724c,Light Gradient Boosting on ElasticNet Predicti...,Tuned,169.924455
3,692fd37102faa1b369c44af6,Ridge Regressor,Original,186.539415
4,692fd9bad89a0b04b789a1b8,Ridge Regressor,Tuned,186.539415



 Predicting with Model 692fd37102faa1b369c44afa (Original)
> Predictions added for c44afa_Original

 Predicting with Model 692fd952ad0f5df96189a0b3 (Tuned)
> Predictions added for 89a0b3_Tuned

 Predicting with Model 692fd8fe58440a870d56724c (Tuned)
> Predictions added for 56724c_Tuned

 Predicting with Model 692fd37102faa1b369c44af6 (Original)
> Predictions added for c44af6_Original

 Predicting with Model 692fd9bad89a0b04b789a1b8 (Tuned)
> Predictions added for 89a1b8_Tuned

 MODEL–PREDICTION COLUMN MAP:


,model_id,model_name,type,prediction_column
0,692fd37102faa1b369c44afa,Light Gradient Boosting on ElasticNet Predicti...,Original,c44afa_Original_pred
1,692fd952ad0f5df96189a0b3,Light Gradient Boosting on ElasticNet Predicti...,Tuned,89a0b3_Tuned_pred
2,692fd8fe58440a870d56724c,Light Gradient Boosting on ElasticNet Predicti...,Tuned,56724c_Tuned_pred
3,692fd37102faa1b369c44af6,Ridge Regressor,Original,c44af6_Original_pred
4,692fd9bad89a0b04b789a1b8,Ridge Regressor,Tuned,89a1b8_Tuned_pred



 Final predictions saved to: /home/notebooks/storage/Results/Final_Scoring_Top5.csv

============================ SCORING PHASE COMPLETE ============================


 Prediction columns generated:

['c44afa_Original_pred', '89a0b3_Tuned_pred', '56724c_Tuned_pred', 'c44af6_Original_pred', '89a1b8_Tuned_pred']


In [67]:
predsAll.head()

,Apolice,ID_entidade,data_cotacao,Data_tarifacao,Data_Anulacao,Flag_Covid,canal,HAS_DECO,Tipo_Fraccionamento,Meio_Outros_Pag,Idade,Antiguidade_Cliente,TS_latitude,TS_longitude,TS_Distrito,TS_NUTS1,TS_NUTS2,TS_NUTS3,TS_regiao_marktest,TS_Interior_litoral,TS_Rural_urbano,Premio_apresentado_apolice,Produto,Idade_Viatura,Flag_Importado_RU,Marca_Viatura,Segmento,cc,netpower,combust,PP,Valor_Viatura,pnovo,Flag_reboque,Flag_garagem,AC_latitude,AC_longitude,AC_Distrito,AC_NUTS1,AC_NUTS2,AC_NUTS3,AC_regiao_marktest,AC_Interior_litoral,AC_Rural_urbano,Flag_Proprietario_viat,CH_Idade,Anos_Carta,Flag_Companhias_Directas,Flag_Companhias_Tradicionais,NCD,N_sinistros_ultimos_3anos,Capital_AV,QIV,CCC,Count_Coberturas,Receitas_Historicos_AU,Custo_Historicos_AU,Saldo_AU,Max_Maturidade_AU,Count_Apl_Vigor_AU,Count_Apl_Anuladas_AU,Meses_desde_ult_sin_AU,Receitas_Historicos_MRH,Custo_Historicos_MRH,Saldo_MRH,Max_Maturidade_MRH,Count_Apl_Vigor_MRH,Count_Apl_Anuladas_MRH,Meses_desde_ult_sin_MRH,Cotacoes_1m,Cotacoes_3m,Cotacoes_12m,Avg_annual_quotes,Online_quote_ratio,Meses_desde_ult_cot,Viaturas_Distintas,Inbound_calls_atend_1m,Inbound_calls_atend_3m,Inbound_calls_atend_12m,Avg_inbound_yr_calls,Outbound_calls_atend_1m,Outbound_calls_atend_3m,Outbound_calls_atend_12m,Avg_outbound_yr_calls,Receitas_1,Receitas_2,Custos_acquisition_1,Custos_vouchers_1,Custos_claims_1,Custos_claims_2,Custos_cares_1,Custos_cares_2,Custos_1,Custos_2,CLV_1,actuals,c44afa_Original_pred,89a0b3_Tuned_pred,56724c_Tuned_pred,c44af6_Original_pred,89a1b8_Tuned_pred
0,960488739,4001124058,27-04-2020,28-04-2020,24-09-2020,1,CALL CENTER,0,Trimestral,Multibanco,43,9,38.72082,-9.35069,LISBOA,PORTUGAL CONTINENTAL,REGI?O DE LISBOA,?REA METROPOLITANA DE LISBOA,REGI?O DA GRANDE LISBOA,LITORAL,?REA PREDOMINANTEMENTE URBANA,180.83,SAFE,7,1,SEAT,LIGEIRO,1199,55,D,15.13,7550,17617,0,0,38.72082,-9.35069,LISBOA,PORTUGAL CONTINENTAL,REGI?O DE LISBOA,?REA METROPOLITANA DE LISBOA,REGI?O DA GRANDE LISBOA,LITORAL,?REA PREDOMINANTEMENTE URBANA,1.0,43,23,1,1,18,0,ok 2,0,0,0,401.62,0.00,401.62,2,0,2,NaN,0.0,0,0.0,0,0,0,NaN,1,1,1,1.0,1.0,0.0,1.0,0,0,0,0.0,4,4,4,4.0,"66,46",NaN,38.19,NaN,NaN,NaN,NaN,NaN,38.19,NaN,28.27,28.27,-441.534250,-443.733920,-440.827411,-24.661221,-24.661221
1,960432583,4001404194,22-02-2019,23-02-2019,23-02-2020,0,CALL CENTER,0,Trimestral,D??bito Direto,36,0,38.81883,-9.12226,LISBOA,PORTUGAL CONTINENTAL,REGI?O DE LISBOA,?REA METROPOLITANA DE LISBOA,REGI?O DA GRANDE LISBOA,INTERIOR,?REA PREDOMINANTEMENTE URBANA,257.33,SAFE,20,0,RENAULT,LIGEIRO,1149,43,G,14.67,0,9896,0,0,38.81883,-9.12226,LISBOA,PORTUGAL CONTINENTAL,REGI?O DE LISBOA,?REA METROPOLITANA DE LISBOA,REGI?O DA GRANDE LISBOA,INTERIOR,?REA PREDOMINANTEMENTE URBANA,0.0,36,12,1,0,12,0,ok 2,0,0,0,0.00,0.00,0.00,0,0,0,NaN,0.0,0,0.0,0,0,0,NaN,2,2,2,2.0,1.0,0.0,1.0,0,0,0,0.0,4,4,4,4.0,"227,52",NaN,38.19,NaN,"1.762,30",NaN,"59,11",NaN,1859.60,NaN,-1632.08,-1632.08,-2049.550506,-2041.408141,-2040.393866,-1640.259287,-1640.259287
2,960521014,4001472749,24-11-2020,02-12-2020,02-12-2025,1,INTERNET,0,Anual,D??bito Direto,73,0,41.22965,-8.37874,PORTO,PORTUGAL CONTINENTAL,REGI?O DO NORTE,?REA METROPOLITANA DO PORTO,REGI?O DO INTERIOR NORTE,INTERIOR,?REA PREDOMINANTEMENTE URBANA,538.12,PREMIUM,9,0,MERCEDES-BENZ,LIGEIRO,2143,150,D,8.50,19600,58989,0,1,41.22965,-8.37874,PORTO,PORTUGAL CONTINENTAL,REGI?O DO NORTE,?REA METROPOLITANA DO PORTO,REGI?O DO INTERIOR NORTE,INTERIOR,?REA PREDOMINANTEMENTE URBANA,1.0,73,51,0,1,18,0,ok 2,1,1,5,0.00,0.00,0.00,0,0,0,NaN,0.0,0,0.0,0,0,0,NaN,1,1,1,1.0,1.0,0.0,1.0,0,0,0,0.0,0,0,0,0.0,"479,38","476,69",26.25,NaN,NaN,NaN,NaN,NaN,26.25,NaN,453.13,929.82,346.624765,358.551655,362.968747,800.645490,800.645490
3,960468477,4001161881,18-11-2019,18-11-2019,13-06-2023,0,CALL CENTER,0,Trimestral,Multibanco,42,7,38.91609,-9.02767,LISBOA,PORTUGAL CONTINENTAL,REGI?O DE LISBOA,?REA METROPOLITANA DE LISBOA,REGI?O DO LITORAL CENTRO,INTERIOR,?REA MEDIAMENTE URBANA,381.36,PREMIUM,7,1,PEUGEOT,STATIO